In [1]:
import json
import ipywidgets as widgets
from IPython.display import display, Image
import os
from Qwen2VL.processing_qwen2_vl import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info
import ast  # To safely evaluate JSON-like strings
from PIL import Image, ImageDraw
import io

In [2]:
def draw_click_point(img_path, click_x, click_y, bbox):
    if os.path.exists(img_path):
        img = Image.open(img_path)
        w, h = img.size  # Get image dimensions
        
        # Convert relative to absolute coordinates
        abs_x = int(click_x * w)
        abs_y = int(click_y * h)
        
        # Draw the click point (dot)
        draw = ImageDraw.Draw(img)
        dot_radius = 10  # Adjust dot size if needed
        draw.ellipse(
            (abs_x - dot_radius, abs_y - dot_radius, abs_x + dot_radius, abs_y + dot_radius),
            fill="red"
        )

        # Draw the bounding box if provided
        if bbox:
            bbox_x = bbox.get("x", 0)
            bbox_y = bbox.get("y", 0)
            bbox_w = bbox.get("width", 0)
            bbox_h = bbox.get("height", 0)
            bbox_coords = [(bbox_x, bbox_y), (bbox_x + bbox_w, bbox_y + bbox_h)]
            draw.rectangle(bbox_coords, outline="blue", width=3)  # Blue bounding box

        # Write image to a bytes buffer instead of saving to disk
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        image_data = buffer.getvalue()
        buffer.close()
        return image_data
    else:
        return None

In [3]:
# Load JSON file
# json_file = "/home/syc/intern/wanshan/Qwen2-VL/agent_tasks/Qwen2-VL-2B/qwen2vl_mind2web_website.json"  # Change this to your JSON file path
json_file = "/home/syc/intern/wanshan/Qwen2-VL/agent_tasks/qwen2vl_resampler_keep_0.8_website.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
# Filter instances where Ele_match is False
filtered_data = [
    [step for step in episode] 
    for episode in data
]

# Initialize index
episode_index = 0
step_index = 0

In [4]:
ground_truth_json_path = "/data/data1/syc/intern/wanshan/mind2map_dataset/mind2web_data_test_website.json"
with open(ground_truth_json_path, "r", encoding="utf-8") as f:
    data = json.load(f)
# Filter instances where Ele_match is False
ground_truth_data = [
    [step for step in episode['actions']] 
    for episode in data
]

In [5]:
navigation_sequence  = []
for episode_idx, episode in enumerate(filtered_data):
    for step_idx, step in enumerate(episode):
        if step['Ele_match'] == False: # visualize Element mismatched
            navigation_sequence.append((episode_idx, step_idx))
sequence_index = 0

In [6]:
len(navigation_sequence)

928

In [12]:
# Function to update display
def update_display():
    global sequence_index
    
    if sequence_index < len(navigation_sequence):
        episode_index, step_index = navigation_sequence[sequence_index]
        episode = filtered_data[episode_index]
        episode_ref = ground_truth_data[episode_index]
        if step_index < len(episode):
            step = episode[step_index]
            step_ref = episode_ref[step_index]
            
            # Extract sentence (convert list of JSON strings into readable text)
            # sentence_text = "\n".join(step["sentence"])
            
            # Update instruction and sentence
            # instruction_label.value = f"**Instruction:** {step['instruction']}"
            sentence_label.value = f"**Action(s):** {step}"
            # Extract click coordinates
            try:
                action_data = ast.literal_eval(step["sentence"][0])  # Convert string to dict
                click_x, click_y = action_data.get("click_point", (0, 0))
                # uigraph_click_y += 0.02
            except Exception as e:
                click_x, click_y = 0, 0  # Default to top-left corner on error

            # get bbox from groundtruth
            try:
                bbox = step_ref['bbox']
            except:
                print('no bbox exist')
                return
            
            # Load and update image
            img_path = step["img_path"]
            
            image = draw_click_point(img_path, click_x, click_y, bbox)

            if os.path.exists(img_path):
                image_widget.value = image
            else:
                sentence_label.value += f"\n(Error: Image not found at {img_path})"
        else:
            sentence_label.value = "No more steps in this episode."
            image_widget.value = b""
    else:
        # instruction_label.value = "No more episodes."
        sentence_label.value = ""
        image_widget.value = b""

# Next button function
def next_step(_):
    global sequence_index
    
    # Move to the next item in the navigation sequence
    if sequence_index < len(navigation_sequence) - 1:
        sequence_index += 1
        update_display()


    update_display()

# Widgets
# instruction_label = widgets.HTML()
sentence_label = widgets.HTML()
image_widget = widgets.Image(format='png', width=800)  # Set Image Size

next_button = widgets.Button(description="Next Step")
next_button.on_click(next_step)

# Initial display
update_display()

# Layout
display(sentence_label,  image_widget, next_button)

HTML(value='**Action(s):** {\'annot_id\': \'a6cb6d95-26cb-43c4-8c70-ffd37fb2818a\', \'img_path\': \'/data/data…

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\x00\x00\x00\x02\xd0\x08\x02\x00\x00\x00@\x1fJ\x0…

Button(description='Next Step', style=ButtonStyle())